<a href="https://colab.research.google.com/github/tomonari-masada/course2022-stats2/blob/main/PLSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PLSIの実装例

In [ ]:
!pip install torchdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tqdm import tqdm
import numpy as np
from numpy.random import default_rng
from sklearn.feature_extraction.text import CountVectorizer
from torchtext.datasets import AG_NEWS

rng = default_rng()

* https://pytorch.org/text/stable/datasets.html

In [ ]:
train_iter = AG_NEWS(split='train')

def tokenize(line):
  return line.split()

corpus = []
for label, line in tqdm(train_iter):
  corpus.append(line.lower())
  if len(corpus) >= 20000:
    break

19999it [00:00, 25267.06it/s]


In [ ]:
corpus[0]

"wall st. bears claw back into the black (reuters) reuters - short-sellers, wall street's dwindling\\band of ultra-cynics, are seeing green again."

In [ ]:
vectorizer = CountVectorizer(stop_words='english', min_df=0.001, max_df=0.1)
X = vectorizer.fit_transform(corpus).toarray()
vocab = vectorizer.get_feature_names_out()

* 全ての単語が削除されてしまっている文書がないことを確認
 * このチェックをしておかないと、ゼロで割ることによるエラーが出るため。

In [ ]:
(X.sum(-1) == 0).sum()

0

In [ ]:
N, W = X.shape
print(f"{N} docs, {W} different words")

20000 docs, 3959 different words


* トピック数の設定とパラメータの初期化

In [ ]:
K=10
theta = rng.random([len(corpus), K])
theta = theta / theta.sum(1, keepdims=True)
phi = rng.random([K, len(vocab)])
phi = phi / phi.sum(1, keepdims=True)

In [ ]:
mb_size = 1000
for iter in range(200):
  new_theta = theta.copy()
  new_phi = np.zeros_like(phi)
  for i in tqdm(range(1, len(corpus) + 1, mb_size)):
    q = theta[i:i+mb_size,np.newaxis,:] * phi.transpose()[np.newaxis,:,:]
    pseudo_counts = q * X[i:i+mb_size,:,np.newaxis]
    temp_theta = pseudo_counts.sum(1)
    temp_theta = temp_theta / temp_theta.sum(1, keepdims=True)
    new_theta[i:i+mb_size,:] = temp_theta
    new_phi += pseudo_counts.sum(0).transpose()
  theta = new_theta
  phi = new_phi / new_phi.sum(1, keepdims=True)
  if (iter + 1) % 10 == 0:
    for k in range(K):
      print(' '.join(list(vocab[(- phi).argsort()[k,:20]])))
    print('-' * 80)

100%|██████████| 20/20 [00:05<00:00,  3.41it/s]


company yesterday microsoft olympic gold security athens china com software internet medal corp windows united saturday thursday today state market
week friday athens olympic afp gold says team win microsoft iraq president york years night lt al tuesday court gt
year monday thursday government athens million president world olympic market tuesday bank percent york time sales second group report minister
quot oil stocks company prices york monday united today tuesday gt friday world google high day lt corp says open
ap thursday monday wednesday time night olympic microsoft athens tuesday internet second people iraq gold world company friday officials government
lt gt tuesday world ap fullquote stocks com company year wednesday york http day ticker investor target million washington iraq
world iraq york sunday second thursday oil day najaf people prices monday group city lt afp president al iraqi record
ap time tuesday iraq million games olympic years gold monday game sunday president bu

100%|██████████| 20/20 [00:05<00:00,  3.36it/s]


company yesterday microsoft security software windows internet today corp com china market united apple computer music million state saturday google
athens olympic week friday gold afp team says win years medal president night games olympics men united court greece al
year monday government million percent old president thursday sales second bank research market world report profit time tuesday york end
quot company world says united president today yesterday oil athens gt friday monday olympic microsoft lt iraq time high china
ap monday thursday wednesday night tuesday time friday game iraq people officials internet bush microsoft team second olympic saturday world
gt lt tuesday fullquote stocks com http ticker href target investor www quickinfo aspx york world font washington corp company
world iraq york thursday sunday second people najaf day city group iraqi al killed government monday president record win prices
ap time tuesday bush iraq monday sunday president wednesday game year

100%|██████████| 20/20 [00:06<00:00,  3.05it/s]


...

company microsoft yesterday software security windows today corp internet service google million market apple computer business com xp music update
athens olympic gold week friday afp medal team olympics games united says men win years night greece saturday president american
year monday old percent million profit government second sales president end market quarter research time billion world thursday united talks
quot says president athens olympic company world yesterday microsoft united time iraq today gold night group games lt gt minister
ap monday wednesday night friday thursday bush game tuesday time saturday sunday victory officials 151 team iraq president year people
gt lt fullquote tuesday http stocks com www href ticker target investor quickinfo aspx york font corp profit washington face
world iraq thursday york sunday people najaf second day al iraqi city killed government group tuesday end cup police cleric
ap time bush tuesday saturday president wednesday night season mond

100%|██████████| 20/20 [00:05<00:00,  3.51it/s]


000 pop pope popular population porn port portable portfolio position positions positive possibility poor possible post posted posting postponed posts
000 pop pope popular population porn port portable portfolio position positions positive possibility poor possible post posted posting postponed posts
000 pop pope popular population porn port portable portfolio position positions positive possibility poor possible post posted posting postponed posts
000 pop pope popular population porn port portable portfolio position positions positive possibility poor possible post posted posting postponed posts
000 pop pope popular population porn port portable portfolio position positions positive possibility poor possible post posted posting postponed posts
000 pop pope popular population porn port portable portfolio position positions positive possibility poor possible post posted posting postponed posts
000 pop pope popular population porn port portable portfolio position positions positive possi

 75%|███████▌  | 15/20 [00:04<00:01,  3.08it/s]


KeyboardInterrupt: ignored

In [ ]:
for k in range(K):
  print(' '.join(list(vocab[(- phi).argsort()[k,:30]])))
print('-' * 80)